In [ ]:
import numpy as np
import pandas as pd
import os
import re
import networkx as nx
import pickle
from numpy import genfromtxt

In [ ]:
data_dir = '/home/ubuntu/Stella/MLVU_multimodality/build functional graph/valid_fMRI_con_mat_1210'

In [ ]:
targ_folder = os.listdir(data_dir)

In [ ]:
con_mat_list=list(targ_folder)

In [ ]:
len(con_mat_list)

In [ ]:
con_mat_list[0]

In [ ]:
node_skeleton = pd.read_csv('fMRI - sMRI.csv')

In [ ]:
node_data = pd.read_csv('mor.some.qc.csv')

In [ ]:
node_data['subjectkey'] = node_data['subjectkey'].apply(lambda x : x.replace('_','')) #remove _ from subjectkey

In [ ]:
node_skeleton_array  = np.array(node_skeleton.iloc[:, 3:])

In [ ]:
node_skeleton_array.shape

In [ ]:
node_skeleton_array[9][1]

In [ ]:
def node_feat_func(subject_id):
    node_feat = list()
    data = node_data[node_data['subjectkey'] == subject_id]
    for i in range (51):
        for j in range(2):
            col_name = node_skeleton_array[i][j]
            if pd.isna(col_name) == True:
                node_feat.append(0)
            else:
                node_feat.append(data[col_name].values[0])
    test = (np.array(node_feat)).reshape(51,2)
    # make pre_ans (col: thickness, volume, row: 51)
    pre_ans = np.zeros((51, 1))
    for k in range (51):
        pre_ans[k] = float(test[k][0])+float(test[k][1]) #volume
    # make final ans (col: thickness, volume, row:48)
    ans = np.zeros((51, 1))
    for m in range(51):
        if m == 3 or m == 29 or m == 34 :
            ans[m] = pre_ans[m] + pre_ans[m+1]#add
        elif m==4 or m==30 or m==35:
            ans[m][0] = 1.00 #replace with 0
        else:
            ans[m][0] = pre_ans[m][0]
        final = np.delete(ans,(4, 30, 35), axis = 0)
                                         
    # change format
    keylist=[]
    valuelist=[]

    for j,k in enumerate(final):
        keylist.append(str(j))
        valuelist.append(str(int(k)))
    node_feat_dict=dict(zip(keylist, valuelist)) 
    return node_feat_dict

In [ ]:
test = node_feat_func('NDARINVGL08BW0W')

In [ ]:
test

In [ ]:
edge_index.shape

In [ ]:
def edge_list_func(adjacency):
    edge_list=[]
    for i in range (48):
        for j in range (48):
            edge_list.append((i, j, adjacency[i][j]))
                
    edge_new_list=[]
    for i in range (len(edge_list)):
        if edge_list[i][2] >= 0.2:
            edge_new_list.append(edge_list[i])
            
    return edge_new_list

In [ ]:
def num_edges_func(edge_list):
    num_edges=len(edge_list)
    return num_edges

In [ ]:
def edge_feat_func(num_edges, edge_list):
    # initialization
    edgefeat_dim = 1
    edge_feat = np.zeros((int(num_edges), edgefeat_dim))
    edge_index = np.zeros((int(num_edges),2))
    # calculation
    for i in range (num_edges):
        source = edge_list[i][0]
        target = edge_list[i][1]
        edge_index[i][0] = int(source)
        edge_index[i][1] = int(target)
    # change format
    edge_index_list=[]
    for i in range(edge_index.shape[0]):
        source = int(edge_index[i][0])
        target = int(edge_index[i][1])
        edge_index_list.append([source, target])
    return edge_index_list

In [ ]:
for i in range(len(con_mat_list)):
    # load data
    data = os.path.join(data_dir, con_mat_list[i])
    subject_id = con_mat_list[i].split('-')[0]
    
    # make adjacency matrix
    adjacency = np.load(data)
    
    # edge_index
    edge_list = edge_list_func(adjacency)
    num_edges = num_edges_func(edge_list)
    edge_index = edge_feat_func(num_edges, edge_list)
    
    # node_feat
    node_feat = node_feat_func(subject_id)
    
    # draw graph
    graph = dict({'edges': edge_index, 'features': node_feat})
    
    print(graph)
    
    with open('/home/ubuntu/Stella/MLVU_multimodality/build functional graph/functional_graph_for_graph2vec_1212/graph2vec_functional_graph_%s.json'%subject_id, 'wb') as file:
        pickle.dump(graph, file)